In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from buck.analysis.basics import ingest_images

# Your existing ingestion
fpath = "C:\\Users\\aaron\\Dropbox\\AI Projects\\buck\\images\\squared\\color\\*.png"
images, ages = ingest_images(fpath)
print(len(images),'images found')

In [ ]:
from buck.analysis.basics import split_data

Xtr_og, ytr_og, Xval, yval, Xte, yte_onehot, ages, l_map = split_data(images, ages)

In [ ]:
from buck.analysis.basics import homogenize_data

augment_multiplier = 30
X_train, y_train, X_test, y_true, label_mapping, num_classes = homogenize_data(Xtr_og, ytr_og, Xte,yte_onehot, l_map, augment_multiplier)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import timm
import numpy as np
import time
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

class OptimizedEfficientNetTester:
    """Optimized EfficientNet training with better hyperparameters"""
    
    def __init__(self, num_classes=5):
        self.num_classes = num_classes
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"🚀 Optimized EfficientNet Tester")
        print(f"   Device: {self.device}")
        print(f"   Classes: {num_classes}")
    
    def create_optimized_model(self, model_name, freeze_backbone=True):
        """Create EfficientNet with optimized settings"""
        try:
            print(f"   🔧 Creating optimized {model_name}...")
            
            # Create model
            model = timm.create_model(model_name, pretrained=True, num_classes=self.num_classes)
            
            # Option 1: Freeze backbone, only train classifier
            if freeze_backbone:
                print(f"      🧊 Freezing backbone layers...")
                for name, param in model.named_parameters():
                    if 'classifier' not in name and 'head' not in name:
                        param.requires_grad = False
                
                trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
                total_params = sum(p.numel() for p in model.parameters())
                print(f"      ✅ Loaded: {total_params:,} total, {trainable_params:,} trainable")
            else:
                total_params = sum(p.numel() for p in model.parameters())
                print(f"      ✅ Loaded: {total_params:,} parameters (all trainable)")
            
            model = model.to(self.device)
            return model
            
        except Exception as e:
            print(f"      ❌ Failed: {str(e)[:50]}...")
            return None
    
    def train_optimized(self, model, arch_name, train_loader, val_loader, test_loader):
        """Optimized training with better hyperparameters"""
        print(f"   🚀 Training {arch_name} (OPTIMIZED)...")
        
        criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Label smoothing helps
        
        # Better optimizer settings
        optimizer = optim.AdamW(
            model.parameters(), 
            lr=0.01,  # Higher learning rate for faster convergence
            weight_decay=0.01,
            betas=(0.9, 0.999)
        )
        
        # Cosine annealing scheduler
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=1e-6)
        
        best_val_acc = 0.0
        patience_counter = 0
        patience = 8  # More patience
        max_epochs = 25  # More epochs
        
        print(f"      📊 Training setup: {max_epochs} max epochs, patience={patience}")
        
        for epoch in range(max_epochs):
            # Training phase
            model.train()
            train_correct = 0
            train_total = 0
            train_loss = 0.0
            
            for batch_idx, (images, labels) in enumerate(train_loader):
                images, labels = images.to(self.device), labels.to(self.device)
                
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                
                # Gradient clipping to prevent exploding gradients
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                
                optimizer.step()
                
                train_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()
            
            train_acc = 100 * train_correct / train_total
            avg_train_loss = train_loss / len(train_loader)
            
            # Validation phase
            model.eval()
            val_correct = 0
            val_total = 0
            val_loss = 0.0
            
            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(self.device), labels.to(self.device)
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()
                    
                    _, predicted = torch.max(outputs.data, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()
            
            val_acc = 100 * val_correct / val_total
            avg_val_loss = val_loss / len(val_loader)
            
            # Update learning rate
            scheduler.step()
            current_lr = scheduler.get_last_lr()[0]
            
            # Early stopping logic
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                patience_counter = 0
                best_model_state = model.state_dict().copy()
                improvement = "🔥"
            else:
                patience_counter += 1
                improvement = ""
            
            # More frequent progress updates
            if epoch % 2 == 0 or epoch < 5 or improvement:
                gap = train_acc - val_acc
                print(f"      Epoch {epoch:2d}: Train {train_acc:.1f}%, Val {val_acc:.1f}% (gap: {gap:+.1f}%), LR: {current_lr:.2e} {improvement}")
            
            # Early stopping
            if patience_counter >= patience:
                print(f"      Early stopping at epoch {epoch}")
                break
        
        # Restore best model and evaluate on test
        model.load_state_dict(best_model_state)
        
        # Test evaluation
        model.eval()
        test_correct = 0
        test_total = 0
        
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()
        
        test_acc = 100 * test_correct / test_total
        
        print(f"      🎯 {arch_name} FINAL: Val {best_val_acc:.1f}%, Test {test_acc:.1f}%")
        
        return best_val_acc, test_acc
    
    def test_efficientnet_series(self, train_loader, val_loader, test_loader):
        """Test EfficientNet B0-B7 with optimized training"""
        print("🚀 OPTIMIZED EFFICIENTNET B0-B7 TESTING")
        print("="*70)
        print("🎯 IMPROVEMENTS:")
        print("   • Higher learning rate (0.01 vs 0.001)")
        print("   • Cosine annealing scheduler") 
        print("   • Label smoothing (0.1)")
        print("   • Gradient clipping")
        print("   • More epochs (25 vs 12)")
        print("   • More patience (8 vs 4)")
        print("   • Option to freeze backbone")
        print("="*70)
        
        # EfficientNet models to test
        models = [
            ('EfficientNet-B0', 'efficientnet_b0'),
            ('EfficientNet-B1', 'efficientnet_b1'),
            ('EfficientNet-B2', 'efficientnet_b2'),
            ('EfficientNet-B3', 'efficientnet_b3'),
            ('EfficientNet-B4', 'efficientnet_b4'),
        ]
        
        results = []
        
        for i, (arch_name, model_name) in enumerate(models, 1):
            print(f"\n[{i}/{len(models)}] 🎯 OPTIMIZED {arch_name}")
            print("-" * 60)
            
            start_time = time.time()
            
            # Test with frozen backbone first
            print("   🧊 FROZEN BACKBONE VERSION:")
            model_frozen = self.create_optimized_model(model_name, freeze_backbone=True)
            
            if model_frozen is not None:
                try:
                    val_acc_frozen, test_acc_frozen = self.train_optimized(
                        model_frozen, f"{arch_name}-Frozen", train_loader, val_loader, test_loader
                    )
                    
                    results.append({
                        'name': f"{arch_name}-Frozen",
                        'val_accuracy': val_acc_frozen,
                        'test_accuracy': test_acc_frozen,
                        'training_time': time.time() - start_time
                    })
                    
                except Exception as e:
                    print(f"      ❌ Frozen training failed: {str(e)[:50]}...")
            
            # Test with full fine-tuning if frozen version works well
            if 'val_acc_frozen' in locals() and val_acc_frozen > 40:
                print(f"   🔥 FULL FINE-TUNING VERSION (frozen got {val_acc_frozen:.1f}%):")
                model_full = self.create_optimized_model(model_name, freeze_backbone=False)
                
                if model_full is not None:
                    try:
                        # Use lower learning rate for full fine-tuning
                        optimizer_full = optim.AdamW(model_full.parameters(), lr=0.001, weight_decay=0.01)
                        
                        val_acc_full, test_acc_full = self.train_optimized(
                            model_full, f"{arch_name}-FullFT", train_loader, val_loader, test_loader
                        )
                        
                        results.append({
                            'name': f"{arch_name}-FullFT", 
                            'val_accuracy': val_acc_full,
                            'test_accuracy': test_acc_full,
                            'training_time': time.time() - start_time
                        })
                        
                    except Exception as e:
                        print(f"      ❌ Full fine-tuning failed: {str(e)[:50]}...")
            
            print(f"   ⏱️ Total time for {arch_name}: {time.time() - start_time:.1f}s")
        
        # Sort results
        results.sort(key=lambda x: x['test_accuracy'], reverse=True)
        
        # Display results
        print(f"\n🏆 OPTIMIZED EFFICIENTNET RESULTS")
        print("="*70)
        print(f"{'Rank':<4} {'Architecture':<25} {'Val%':<8} {'Test%':<8} {'vs 54.2%'}")
        print("-" * 70)
        
        for i, result in enumerate(results, 1):
            val_acc = result['val_accuracy']
            test_acc = result['test_accuracy']
            
            if test_acc >= 70.0:
                status = "🎉 BREAKTHROUGH!"
            elif test_acc > 54.2:
                status = "🔥 NEW BEST!"
            elif test_acc > 45.0:
                status = "📈 Good"
            else:
                status = "📉 Poor"
            
            print(f"{i:<4} {result['name']:<25} {val_acc:<7.1f} {test_acc:<7.1f} {status}")
        
        if results:
            best = results[0]
            print(f"\n🎉 BEST OPTIMIZED EFFICIENTNET:")
            print(f"   🏆 {best['name']}: {best['test_accuracy']:.1f}% test accuracy")
            
            if best['test_accuracy'] > 54.2:
                improvement = best['test_accuracy'] - 54.2
                print(f"   🚀 IMPROVEMENT: +{improvement:.1f}% over baseline!")
            else:
                print(f"   💡 Still below 54.2% baseline - trying other optimizations...")
        
        print("="*70)
        return results

def run_optimized_efficientnet_test(train_loader, val_loader, test_loader):
    """Run optimized EfficientNet testing"""
    tester = OptimizedEfficientNetTester(num_classes=5)
    return tester.test_efficientnet_series(train_loader, val_loader, test_loader)

# To use this with your existing data loaders:
print("🚀 READY FOR OPTIMIZED EFFICIENTNET TESTING!")
print("Run: optimized_results = run_optimized_efficientnet_test(train_loader, val_loader, test_loader)")
optimized_results = run_optimized_efficientnet_test(train_loader, val_loader, test_loader)

In [ ]:
# CUDA Debugger

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import timm
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

def test_cuda_setup():
    """Test CUDA setup thoroughly"""
    print("🔍 CUDA DIAGNOSTICS:")
    print(f"   CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"   CUDA device count: {torch.cuda.device_count()}")
        print(f"   Current device: {torch.cuda.current_device()}")
        print(f"   Device name: {torch.cuda.get_device_name()}")
        print(f"   Device memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
        
        # Test basic tensor operations
        print("   Testing GPU tensor operations...")
        try:
            x = torch.randn(100, 100).cuda()
            y = torch.randn(100, 100).cuda()
            z = torch.mm(x, y)
            print("   ✅ Basic GPU operations working")
            
            # Check memory usage
            print(f"   GPU memory allocated: {torch.cuda.memory_allocated() / 1e9:.3f} GB")
            print(f"   GPU memory cached: {torch.cuda.memory_reserved() / 1e9:.3f} GB")
            
        except Exception as e:
            print(f"   ❌ GPU test failed: {e}")
            return False
    else:
        print("   ❌ CUDA not available - will use CPU")
        return False
    
    return True

class DebugDeerDataset(Dataset):
    """Debug version with extensive logging"""
    
    def __init__(self, X, y, is_training=True):
        print(f"🔍 Creating dataset...")
        print(f"   Input X type: {type(X)}, shape: {X.shape if hasattr(X, 'shape') else 'unknown'}")
        print(f"   Input y type: {type(y)}, shape: {y.shape if hasattr(y, 'shape') else 'unknown'}")
        
        # Convert to tensors
        if isinstance(X, np.ndarray):
            print("   Converting X from numpy to tensor...")
            X = torch.FloatTensor(X)
        if isinstance(y, np.ndarray):
            print("   Converting y from numpy to tensor...")
            y = torch.LongTensor(y)
            
        self.X = X
        self.y = y
        self.is_training = is_training
        
        print(f"   Final X tensor shape: {X.shape}")
        print(f"   Final y tensor shape: {y.shape}")
        print(f"   Dataset length: {len(X)}")
        print(f"   Is training: {is_training}")
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if idx == 0:  # Log first item details
            print(f"🔍 Getting first item (idx={idx})...")
            
        image = self.X[idx].clone()
        label = self.y[idx]
        
        if idx == 0:
            print(f"   Original image shape: {image.shape}")
            print(f"   Image dtype: {image.dtype}")
            print(f"   Image min/max: {image.min():.3f}/{image.max():.3f}")
            print(f"   Label: {label}")
        
        # Normalize
        if image.max() > 1.0:
            image = image / 255.0
            
        # Permute to CHW
        image = image.permute(2, 0, 1)  # (3, 288, 288)
        
        if idx == 0:
            print(f"   After permute shape: {image.shape}")
        
        # Simple augmentation
        if self.is_training and torch.rand(1) > 0.5:
            image = torch.flip(image, dims=[2])
            
        # Resize to 224x224
        image = image.unsqueeze(0)  # (1, 3, 288, 288)
        image = F.interpolate(image, size=(224, 224), mode='bilinear', align_corners=False)
        image = image.squeeze(0)  # (3, 224, 224)
        
        if idx == 0:
            print(f"   After resize shape: {image.shape}")
        
        # Normalize
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        image = (image - mean) / std
        
        if idx == 0:
            print(f"   Final normalized shape: {image.shape}")
            print(f"   Final min/max: {image.min():.3f}/{image.max():.3f}")
        
        return image, label

class DebugEfficientNet:
    """Debug version with step-by-step logging"""
    
    def __init__(self, num_classes=5):
        print(f"🔍 Creating EfficientNet model...")
        self.num_classes = num_classes
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"   Device: {self.device}")
        
        self.create_model()
        
    def create_model(self):
        print("   Loading EfficientNet-B0...")
        try:
            self.model = timm.create_model(
                'efficientnet_b0',
                pretrained=True,
                num_classes=self.num_classes,
                drop_rate=0.1
            )
            print("   ✅ Model created successfully")
            
            print("   Moving model to device...")
            self.model = self.model.to(self.device)
            print("   ✅ Model moved to device")
            
            # Test model with dummy input
            print("   Testing model with dummy input...")
            dummy_input = torch.randn(2, 3, 224, 224).to(self.device)
            with torch.no_grad():
                dummy_output = self.model(dummy_input)
            print(f"   ✅ Model test successful. Output shape: {dummy_output.shape}")
            
            param_count = sum(p.numel() for p in self.model.parameters())
            print(f"   Model parameters: {param_count:,}")
            
        except Exception as e:
            print(f"   ❌ Model creation failed: {e}")
            raise e
    
    def test_dataloader(self, train_loader):
        """Test if dataloader works"""
        print("🔍 Testing dataloader...")
        try:
            print("   Getting first batch...")
            start_time = time.time()
            
            for batch_idx, (images, labels) in enumerate(train_loader):
                print(f"   Batch {batch_idx}: images {images.shape}, labels {labels.shape}")
                print(f"   Images dtype: {images.dtype}, Labels dtype: {labels.dtype}")
                print(f"   Moving to device...")
                
                images = images.to(self.device)
                labels = labels.to(self.device)
                
                print(f"   ✅ Successfully moved batch to {self.device}")
                print(f"   Memory allocated: {torch.cuda.memory_allocated() / 1e9:.3f} GB")
                
                # Test model forward pass
                print("   Testing model forward pass...")
                start_forward = time.time()
                outputs = self.model(images)
                forward_time = time.time() - start_forward
                
                print(f"   ✅ Forward pass successful in {forward_time:.3f}s")
                print(f"   Output shape: {outputs.shape}")
                print(f"   Memory after forward: {torch.cuda.memory_allocated() / 1e9:.3f} GB")
                
                if batch_idx >= 1:  # Test 2 batches
                    break
                    
            total_time = time.time() - start_time
            print(f"   ✅ Dataloader test completed in {total_time:.3f}s")
            return True
            
        except Exception as e:
            print(f"   ❌ Dataloader test failed: {e}")
            import traceback
            traceback.print_exc()
            return False
    
    def prepare_data_loaders(self, X_train, y_train, X_val, y_val, batch_size=16):
        print("🔍 Preparing data loaders...")
        
        train_dataset = DebugDeerDataset(X_train, y_train, is_training=True)
        val_dataset = DebugDeerDataset(X_val, y_val, is_training=False)
        
        print(f"   Creating train loader with batch_size={batch_size}...")
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                                num_workers=0, drop_last=True)
        
        print(f"   Creating val loader...")
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, 
                              num_workers=0)
        
        print(f"   ✅ Loaders created:")
        print(f"      Train: {len(train_loader)} batches")
        print(f"      Val: {len(val_loader)} batches")
        
        return train_loader, val_loader
    
    def quick_train_test(self, train_loader, val_loader):
        """Quick training test - just 2 epochs"""
        print("🔍 Quick training test (2 epochs)...")
        
        optimizer = optim.Adam(self.model.parameters(), lr=1e-3)
        criterion = nn.CrossEntropyLoss()
        
        for epoch in range(2):
            print(f"   Epoch {epoch+1}/2...")
            self.model.train()
            
            total_loss = 0
            correct = 0
            total = 0
            
            for batch_idx, (images, labels) in enumerate(train_loader):
                start_batch = time.time()
                
                images, labels = images.to(self.device), labels.to(self.device)
                
                optimizer.zero_grad()
                outputs = self.model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
                
                batch_time = time.time() - start_batch
                
                if batch_idx % 10 == 0:
                    acc = 100. * correct / total
                    print(f"      Batch {batch_idx}: Loss {loss.item():.4f}, Acc {acc:.1f}%, Time {batch_time:.3f}s")
                    print(f"      GPU Memory: {torch.cuda.memory_allocated() / 1e9:.3f} GB")
                
                if batch_idx >= 20:  # Test just 20 batches
                    break
            
            epoch_acc = 100. * correct / total
            epoch_loss = total_loss / min(21, len(train_loader))
            print(f"   ✅ Epoch {epoch+1} completed: Loss {epoch_loss:.4f}, Acc {epoch_acc:.1f}%")
        
        print("   ✅ Quick training test successful!")

def debug_deer_classification(X_train, y_train, X_val, y_val, X_test, y_true, label_mapping):
    """Complete debug run"""
    print("=" * 60)
    print("🔍 DEBUGGING DEER AGE CLASSIFICATION")
    print("=" * 60)
    
    # Step 1: Test CUDA
    cuda_works = test_cuda_setup()
    
    # Step 2: Check data
    print(f"\n🔍 DATA SHAPES:")
    print(f"   Train: {X_train.shape}")
    print(f"   Val: {X_val.shape}")
    print(f"   Test: {X_test.shape}")
    print(f"   Classes: {len(label_mapping)}")
    
    # Step 3: Fix labels
    print(f"\n🔍 FIXING LABELS:")
    if len(y_train.shape) == 2:
        y_train = np.argmax(y_train, axis=1)
        print("   Fixed y_train from one-hot")
    if len(y_val.shape) == 2:
        y_val = np.argmax(y_val, axis=1)
        print("   Fixed y_val from one-hot")
    if len(y_true.shape) == 2:
        y_true = np.argmax(y_true, axis=1)
        print("   Fixed y_true from one-hot")
    
    print(f"   Final label shapes: train {y_train.shape}, val {y_val.shape}, test {y_true.shape}")
    
    # Step 4: Create model
    print(f"\n🔍 CREATING MODEL:")
    classifier = DebugEfficientNet(num_classes=len(label_mapping))
    
    # Step 5: Create data loaders
    print(f"\n🔍 CREATING DATA LOADERS:")
    train_loader, val_loader = classifier.prepare_data_loaders(
        X_train, y_train, X_val, y_val, batch_size=16
    )
    
    # Step 6: Test data loader
    print(f"\n🔍 TESTING DATA LOADING:")
    dataloader_works = classifier.test_dataloader(train_loader)
    
    if dataloader_works:
        # Step 7: Quick training test
        print(f"\n🔍 TESTING TRAINING:")
        classifier.quick_train_test(train_loader, val_loader)
        
        print("\n" + "=" * 60)
        print("🎉 ALL TESTS PASSED! System is working correctly.")
        print("   You can now run full training with confidence.")
        print("=" * 60)
    else:
        print("\n" + "=" * 60)
        print("❌ ISSUE FOUND IN DATA LOADING")
        print("   Check the error messages above to identify the problem.")
        print("=" * 60)

# Run the debug version
debug_deer_classification(X_train, y_train, Xval, yval, X_test, y_true, label_mapping)

In [ ]:
'''
# Corrected model test
import torch
import torch.nn.functional as F
import numpy as np
import timm
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def load_trained_model(model_path='best_deer_efficientnet.pth', num_classes=5):
    """
    Load the saved deer age classification model
    """
    print(f"🔄 Loading trained model from {model_path}...")
    
    # Recreate the model architecture (same as in training)
    model = timm.create_model(
        'efficientnet_b4',
        pretrained=False,  # Don't download pretrained weights
        num_classes=num_classes,
        drop_rate=0.3,
        drop_path_rate=0.2
    )
    
    # Replace classifier (same as in training)
    in_features = model.classifier.in_features
    model.classifier = torch.nn.Sequential(
        torch.nn.Dropout(p=0.4, inplace=True),
        torch.nn.Linear(in_features, num_classes)
    )
    
    # Load the saved weights
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()
    
    print(f"✅ Model loaded successfully on {device}")
    return model

def evaluate_model_correct_preprocessing(model, X_test, y_true, label_mapping):
    """
    Evaluate model with CORRECT preprocessing (no ImageNet normalization)
    """
    print("🎯 EVALUATING MODEL ON TEST SET - CORRECTED PREPROCESSING")
    print("=" * 60)
    
    # Convert one-hot to class indices if needed
    if len(y_true.shape) == 2:
        print(f"Converting one-hot labels {y_true.shape} to class indices...")
        y_true = np.argmax(y_true, axis=1)
        print(f"✅ Labels now: {y_true.shape}")
    
    # Create reverse mapping
    class_to_age = {v: k for k, v in label_mapping.items()}
    
    print(f"Test set size: {len(X_test)} images")
    print(f"Number of classes: {len(label_mapping)}")
    print(f"Age mapping: {label_mapping}")
    print(f"Class to age: {class_to_age}")
    
    # Get device
    device = next(model.parameters()).device
    
    # Make predictions
    model.eval()
    all_predictions = []
    all_probabilities = []
    
    print("\n🔄 Making predictions with CORRECT preprocessing...")
    print("   ✅ Using raw 0-1 normalized pixels (NO ImageNet normalization)")
    
    with torch.no_grad():
        # Process in batches to avoid memory issues
        batch_size = 16
        for i in range(0, len(X_test), batch_size):
            batch_end = min(i + batch_size, len(X_test))
            batch_images = X_test[i:batch_end]
            
            # Convert to tensor - same as training
            batch_tensor = torch.FloatTensor(batch_images).permute(0, 3, 1, 2).to(device)
            
            # Resize to model input size (384x384 for EfficientNet-B4)
            batch_tensor = F.interpolate(batch_tensor, size=(384, 384), mode='bilinear', align_corners=False)
            
            # ✅ NO IMAGENET NORMALIZATION - just use raw 0-1 pixel values
            # This matches what your model was actually trained on!
            
            # Get predictions
            logits = model(batch_tensor)
            probabilities = F.softmax(logits, dim=1)
            predicted_classes = torch.argmax(logits, dim=1)
            
            # Convert to Python lists
            all_predictions.extend(predicted_classes.tolist())
            all_probabilities.extend(probabilities.tolist())
            
            if i % (batch_size * 5) == 0:
                print(f"   Processed {batch_end}/{len(X_test)} images...")
    
    # Convert to numpy arrays
    y_pred = np.array(all_predictions)
    y_proba = np.array(all_probabilities)
    
    print(f"✅ Predictions complete!")
    
    # Calculate metrics
    print("\n📊 CALCULATING METRICS")
    print("-" * 30)
    
    # Accuracy
    accuracy = accuracy_score(y_true, y_pred)
    print(f"🎯 Test Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    
    # F1 Scores
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_weighted = f1_score(y_true, y_pred, average='weighted')
    f1_micro = f1_score(y_true, y_pred, average='micro')
    
    print(f"📈 F1 Score (Macro): {f1_macro:.4f}")
    print(f"📈 F1 Score (Weighted): {f1_weighted:.4f}")
    print(f"📈 F1 Score (Micro): {f1_micro:.4f}")
    
    # Per-class metrics
    print(f"\n📋 DETAILED CLASSIFICATION REPORT:")
    age_labels = [class_to_age[i] for i in range(len(label_mapping))]
    target_names = [f"{age} years" for age in age_labels]
    
    report = classification_report(y_true, y_pred, target_names=target_names, output_dict=True)
    print(classification_report(y_true, y_pred, target_names=target_names))
    
    # Confusion Matrix
    print(f"\n🔍 CONFUSION MATRIX:")
    cm = confusion_matrix(y_true, y_pred)
    
    # Print confusion matrix
    print("Predicted →")
    print("True ↓   ", "  ".join([f"{age:4.1f}" for age in age_labels]))
    for i, age in enumerate(age_labels):
        row_str = " ".join([f"{cm[i][j]:4d}" for j in range(len(age_labels))])
        print(f"{age:4.1f}     {row_str}")
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=[f"{age}y" for age in age_labels],
                yticklabels=[f"{age}y" for age in age_labels])
    plt.title('Confusion Matrix - Deer Age Classification (Corrected)')
    plt.xlabel('Predicted Age')
    plt.ylabel('True Age')
    plt.tight_layout()
    plt.show()
    
    # Age-based accuracy analysis
    print(f"\n🦌 AGE-SPECIFIC ANALYSIS:")
    print("-" * 30)
    
    for class_idx in range(len(label_mapping)):
        age = class_to_age[class_idx]
        
        # Get indices for this age class
        true_class_mask = (y_true == class_idx)
        if np.sum(true_class_mask) > 0:
            class_accuracy = np.sum((y_true == class_idx) & (y_pred == class_idx)) / np.sum(true_class_mask)
            class_count = np.sum(true_class_mask)
            class_f1 = report[f"{age} years"]['f1-score']
            class_precision = report[f"{age} years"]['precision']
            class_recall = report[f"{age} years"]['recall']
            
            print(f"Age {age} years ({class_count} samples):")
            print(f"   Accuracy: {class_accuracy:.3f} ({class_accuracy*100:.1f}%)")
            print(f"   Precision: {class_precision:.3f}")
            print(f"   Recall: {class_recall:.3f}")
            print(f"   F1-Score: {class_f1:.3f}")
    
    # Tolerance-based accuracy
    print(f"\n🎯 TOLERANCE-BASED ACCURACY:")
    print("-" * 30)
    
    # Convert class indices back to actual ages
    y_true_ages = [class_to_age[idx] for idx in y_true]
    y_pred_ages = [class_to_age[idx] for idx in y_pred]
    
    exact_matches = sum(1 for true, pred in zip(y_true_ages, y_pred_ages) if true == pred)
    within_1_year = sum(1 for true, pred in zip(y_true_ages, y_pred_ages) if abs(true - pred) <= 1.0)
    
    exact_accuracy = exact_matches / len(y_true_ages)
    tolerance_accuracy = within_1_year / len(y_true_ages)
    
    print(f"Exact age accuracy: {exact_accuracy:.3f} ({exact_accuracy*100:.1f}%)")
    print(f"Within ±1 year accuracy: {tolerance_accuracy:.3f} ({tolerance_accuracy*100:.1f}%)")
    
    # Show prediction distribution
    print(f"\n📊 PREDICTION DISTRIBUTION:")
    print("-" * 30)
    pred_counts = {}
    for pred in y_pred:
        age = class_to_age[pred]
        pred_counts[age] = pred_counts.get(age, 0) + 1
    
    for age in sorted(pred_counts.keys()):
        count = pred_counts[age]
        print(f"   Predicted {age} years: {count} times ({count/len(y_pred)*100:.1f}%)")
    
    # Show some example predictions
    print(f"\n🔍 SAMPLE PREDICTIONS:")
    print("-" * 30)
    for i in range(min(15, len(y_true))):
        true_age = y_true_ages[i]
        pred_age = y_pred_ages[i]
        confidence = max(y_proba[i])
        status = "✅" if true_age == pred_age else "❌"
        print(f"{status} Sample {i+1}: True {true_age}y → Predicted {pred_age}y (confidence: {confidence:.2%})")
    
    # Return results
    results = {
        'accuracy': accuracy,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted,
        'f1_micro': f1_micro,
        'confusion_matrix': cm,
        'classification_report': report,
        'predictions': y_pred,
        'probabilities': y_proba,
        'exact_accuracy': exact_accuracy,
        'tolerance_accuracy': tolerance_accuracy
    }
    
    return results

def run_corrected_evaluation(X_test, y_true, label_mapping, model_path='best_deer_efficientnet.pth'):
    """
    Complete corrected evaluation pipeline
    """
    print("🦌 DEER AGE CLASSIFICATION: CORRECTED MODEL EVALUATION")
    print("=" * 70)
    print("🔧 Using CORRECT preprocessing (no ImageNet normalization)")
    print("   This matches what your model was actually trained on!")
    
    # Load the trained model
    model = load_trained_model(model_path, num_classes=len(label_mapping))
    
    # Evaluate the model with correct preprocessing
    results = evaluate_model_correct_preprocessing(model, X_test, y_true, label_mapping)
    
    print("\n✅ CORRECTED EVALUATION COMPLETE!")
    print(f"📊 Final Results Summary:")
    print(f"   Test Accuracy: {results['accuracy']:.3f} ({results['accuracy']*100:.1f}%)")
    print(f"   F1 Score (Macro): {results['f1_macro']:.3f}")
    print(f"   F1 Score (Weighted): {results['f1_weighted']:.3f}")
    print(f"   Exact Age Accuracy: {results['exact_accuracy']:.3f} ({results['exact_accuracy']*100:.1f}%)")
    print(f"   Within ±1 Year Accuracy: {results['tolerance_accuracy']:.3f} ({results['tolerance_accuracy']*100:.1f}%)")
    
    return results

# Run the CORRECTED evaluation
results = run_corrected_evaluation(X_test, y_true, label_mapping, 'best_deer_efficientnet.pth')
'''

In [ ]:
# Imaging where the model is looking
'''
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import cv2
from matplotlib.colors import LinearSegmentedColormap
import timm

class GradCAM:
    """
    Grad-CAM implementation for visualizing what features the model focuses on
    """
    
    def __init__(self, model, target_layer_name):
        self.model = model
        self.target_layer_name = target_layer_name
        self.gradients = None
        self.activations = None
        
        # Register hooks
        self.register_hooks()
    
    def register_hooks(self):
        """Register forward and backward hooks to capture gradients and activations"""
        
        def backward_hook(module, grad_input, grad_output):
            self.gradients = grad_output[0]
        
        def forward_hook(module, input, output):
            self.activations = output
        
        # Find the target layer
        target_layer = None
        for name, module in self.model.named_modules():
            if name == self.target_layer_name:
                target_layer = module
                break
        
        if target_layer is None:
            raise ValueError(f"Layer {self.target_layer_name} not found in model")
        
        # Register hooks
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)
    
    def generate_cam(self, image_tensor, class_idx=None):
        """
        Generate Grad-CAM for a given image
        
        Args:
            image_tensor: Input image tensor (1, 3, H, W)
            class_idx: Target class index (if None, uses predicted class)
        
        Returns:
            cam: Grad-CAM heatmap
            prediction: Model prediction
        """
        # Forward pass
        self.model.eval()
        output = self.model(image_tensor)
        
        if class_idx is None:
            class_idx = output.argmax(dim=1).item()
        
        # Backward pass
        self.model.zero_grad()
        target = output[0][class_idx]
        target.backward()
        
        # Get gradients and activations
        gradients = self.gradients[0]  # (C, H, W)
        activations = self.activations[0]  # (C, H, W)
        
        # Calculate weights (global average pooling of gradients)
        weights = torch.mean(gradients, dim=(1, 2))  # (C,)
        
        # Generate CAM
        cam = torch.zeros(activations.shape[1:], dtype=torch.float32)
        for i, w in enumerate(weights):
            cam += w * activations[i]
        
        # Apply ReLU and normalize
        cam = F.relu(cam)
        cam = cam / cam.max() if cam.max() > 0 else cam
        
        return cam.detach().cpu().numpy(), output.detach().cpu().numpy()

def load_trained_model(model_path='best_deer_efficientnet.pth', num_classes=5):
    """
    Load the saved deer age classification model
    
    Args:
        model_path: Path to saved model weights
        num_classes: Number of age classes
    
    Returns:
        model: Loaded model ready for inference
    """
    print(f"🔄 Loading trained model from {model_path}...")
    
    # Recreate the model architecture (same as in training)
    model = timm.create_model(
        'efficientnet_b4',
        pretrained=False,  # Don't download pretrained weights
        num_classes=num_classes,
        drop_rate=0.3,
        drop_path_rate=0.2
    )
    
    # Replace classifier (same as in training)
    in_features = model.classifier.in_features
    model.classifier = torch.nn.Sequential(
        torch.nn.Dropout(p=0.4, inplace=True),
        torch.nn.Linear(in_features, num_classes)
    )
    
    # Load the saved weights
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()
    
    print(f"✅ Model loaded successfully on {device}")
    return model

def visualize_deer_features(model, X_test, y_true, label_mapping, num_images=6):
    """
    Visualize which features the model focuses on for deer age prediction
    
    Args:
        model: Loaded trained model
        X_test: Test images
        y_true: True labels (as class indices, not one-hot)
        label_mapping: Age to class mapping
        num_images: Number of images to visualize
    """
    
    # Convert one-hot to class indices if needed
    if len(y_true.shape) == 2:
        y_true = np.argmax(y_true, axis=1)
    
    # Create reverse mapping
    class_to_age = {v: k for k, v in label_mapping.items()}
    
    # Initialize Grad-CAM (target the last convolutional layer)
    grad_cam = GradCAM(model, target_layer_name='features.8.2.block.2')
    
    # Select random test images
    indices = np.random.choice(len(X_test), num_images, replace=False)
    
    # Create custom colormap for heatmap
    colors = ['blue', 'cyan', 'yellow', 'orange', 'red']
    n_bins = 256
    cmap = LinearSegmentedColormap.from_list('grad_cam', colors, N=n_bins)
    
    # Create subplot
    fig, axes = plt.subplots(2, num_images, figsize=(20, 8))
    if num_images == 1:
        axes = axes.reshape(2, 1)
    
    device = next(model.parameters()).device
    
    for i, idx in enumerate(indices):
        # Prepare image
        original_image = X_test[idx]
        true_age = class_to_age[y_true[idx]]
        
        # Convert to tensor and add batch dimension
        image_tensor = torch.FloatTensor(original_image).permute(2, 0, 1).unsqueeze(0)
        image_tensor = image_tensor.to(device)
        
        # Resize to model input size (384x384 for EfficientNet-B4)
        image_tensor = F.interpolate(image_tensor, size=(384, 384), mode='bilinear', align_corners=False)
        
        # Normalize (ImageNet normalization)
        mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).to(device)
        std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).to(device)
        image_tensor = (image_tensor - mean) / std
        
        # Generate Grad-CAM
        cam, prediction = grad_cam.generate_cam(image_tensor)
        predicted_class = np.argmax(prediction[0])
        predicted_age = class_to_age[predicted_class]
        confidence = F.softmax(torch.tensor(prediction[0]), dim=0)[predicted_class].item()
        
        # Resize CAM to original image size
        cam_resized = cv2.resize(cam, (original_image.shape[1], original_image.shape[0]))
        
        # Display original image
        axes[0, i].imshow(original_image)
        axes[0, i].set_title(f'Original Image\nTrue Age: {true_age} years', fontsize=12)
        axes[0, i].axis('off')
        
        # Display Grad-CAM overlay
        axes[1, i].imshow(original_image)
        axes[1, i].imshow(cam_resized, cmap=cmap, alpha=0.5, vmin=0, vmax=1)
        axes[1, i].set_title(f'Predicted: {predicted_age} years\nConfidence: {confidence:.2%}', fontsize=12)
        axes[1, i].axis('off')
        
        print(f"Image {i+1}: True={true_age}y, Predicted={predicted_age}y, Confidence={confidence:.1%}")
    
    plt.suptitle('Deer Age Classification: Feature Importance Visualization', fontsize=16, y=0.95)
    plt.tight_layout()
    plt.show()
    
    # Create a colorbar legend
    fig, ax = plt.subplots(figsize=(8, 1))
    gradient = np.linspace(0, 1, 256).reshape(1, -1)
    ax.imshow(gradient, aspect='auto', cmap=cmap)
    ax.set_xlim(0, 256)
    ax.set_yticks([])
    ax.set_xticks([0, 64, 128, 192, 256])
    ax.set_xticklabels(['Low', 'Medium-Low', 'Medium', 'Medium-High', 'High'])
    ax.set_xlabel('Feature Importance for Age Prediction')
    ax.set_title('Grad-CAM Heatmap Legend: Red = High Importance, Blue = Low Importance')
    plt.tight_layout()
    plt.show()

def analyze_feature_patterns(model, X_test, y_true, label_mapping, samples_per_age=3):
    """
    Analyze what features the model focuses on for each age group
    """
    # Convert one-hot to class indices if needed
    if len(y_true.shape) == 2:
        y_true = np.argmax(y_true, axis=1)
    
    class_to_age = {v: k for k, v in label_mapping.items()}
    grad_cam = GradCAM(model, target_layer_name='features.8.2.block.2')
    
    print("🔍 ANALYZING FEATURE PATTERNS BY AGE GROUP")
    print("=" * 60)
    
    # Group images by age
    age_groups = {}
    for i, label in enumerate(y_true):
        age = class_to_age[label]
        if age not in age_groups:
            age_groups[age] = []
        age_groups[age].append(i)
    
    device = next(model.parameters()).device
    
    for age in sorted(age_groups.keys()):
        print(f"\n📊 AGE GROUP: {age} YEARS")
        print("-" * 30)
        
        # Sample images from this age group
        indices = np.random.choice(age_groups[age], min(samples_per_age, len(age_groups[age])), replace=False)
        
        cam_averages = []
        
        for idx in indices:
            # Prepare image
            original_image = X_test[idx]
            image_tensor = torch.FloatTensor(original_image).permute(2, 0, 1).unsqueeze(0)
            image_tensor = image_tensor.to(device)
            image_tensor = F.interpolate(image_tensor, size=(384, 384), mode='bilinear', align_corners=False)
            
            # Normalize
            mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).to(device)
            std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).to(device)
            image_tensor = (image_tensor - mean) / std
            
            # Generate CAM
            cam, prediction = grad_cam.generate_cam(image_tensor)
            predicted_class = np.argmax(prediction[0])
            predicted_age = class_to_age[predicted_class]
            confidence = F.softmax(torch.tensor(prediction[0]), dim=0)[predicted_class].item()
            
            cam_averages.append(cam)
            
            print(f"   Sample {len(cam_averages)}: Predicted {predicted_age}y (confidence: {confidence:.1%})")
        
        # Calculate average attention pattern for this age group
        avg_cam = np.mean(cam_averages, axis=0)
        
        # Analyze where the model focuses
        height, width = avg_cam.shape
        
        # Divide image into regions and calculate average attention
        regions = {
            'Top (Head/Antlers)': avg_cam[:height//3, :].mean(),
            'Middle (Body)': avg_cam[height//3:2*height//3, :].mean(), 
            'Bottom (Legs)': avg_cam[2*height//3:, :].mean(),
            'Left Side': avg_cam[:, :width//2].mean(),
            'Right Side': avg_cam[:, width//2:].mean(),
            'Center': avg_cam[height//4:3*height//4, width//4:3*width//4].mean()
        }
        
        print(f"   🎯 Key focus areas:")
        for region, attention in sorted(regions.items(), key=lambda x: x[1], reverse=True):
            print(f"      {region}: {attention:.3f}")

def run_gradcam_analysis(X_test, y_true, label_mapping, model_path='best_deer_efficientnet.pth'):
    """
    Complete pipeline to load model and run Grad-CAM analysis
    
    Args:
        X_test: Test images
        y_true: True labels
        label_mapping: Age to class mapping dict
        model_path: Path to saved model weights
    """
    print("🦌 DEER AGE CLASSIFICATION: GRAD-CAM FEATURE ANALYSIS")
    print("=" * 70)
    
    # Load the trained model
    model = load_trained_model(model_path, num_classes=len(label_mapping))
    
    # Run Grad-CAM visualization
    print("\n📸 Generating Grad-CAM visualizations...")
    visualize_deer_features(model, X_test, y_true, label_mapping, num_images=6)
    
    # Analyze patterns by age group
    print("\n🔍 Analyzing feature patterns by age group...")
    analyze_feature_patterns(model, X_test, y_true, label_mapping, samples_per_age=3)
    
    print("\n✅ Grad-CAM analysis complete!")
    print("\nKey insights to look for:")
    print("🔍 Young deer (1.5-2.5y): Model may focus on body size, facial features")
    print("🔍 Middle-aged (3.5y): Model may focus on antler development, body mass")  
    print("🔍 Older deer (4.5-5.5y): Model may focus on antler size, body structure")
    
    return model

# USAGE: Run this after training to analyze your saved model
model = run_gradcam_analysis(X_test, y_true, label_mapping, 'best_deer_efficientnet.pth')
'''

In [ ]:
#from buck.classifiers.autotune import optimize_all
#
#optimize_all(X_train, y_train, X_test, y_true, cycles=2)